In [1]:
from __future__ import print_function

from miscc.config import cfg, cfg_from_file
from datasets import TextDataset
from trainer import condGANTrainer as trainer

import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
#import argparse
import numpy as np
import easydict
import warnings

import torch
import torchvision.transforms as transforms

In [2]:
dir_path = (os.path.abspath(os.path.join(os.path.realpath("__file__"), './.')))
sys.path.append(dir_path)

In [3]:
"""def parse_args():
    parser = argparse.ArgumentParser(description='Train a AttnGAN network')
    parser.add_argument('--cfg', dest='cfg_file',
                        help='optional config file',
                        default='cfg/bird_attn2.yml', type=str)
    parser.add_argument('--gpu', dest='gpu_id', type=int, default=-1)
    parser.add_argument('--data_dir', dest='data_dir', type=str, default='')
    parser.add_argument('--manualSeed', type=int, help='manual seed')
    args = parser.parse_args()
    return args
    """

"def parse_args():\n    parser = argparse.ArgumentParser(description='Train a AttnGAN network')\n    parser.add_argument('--cfg', dest='cfg_file',\n                        help='optional config file',\n                        default='cfg/bird_attn2.yml', type=str)\n    parser.add_argument('--gpu', dest='gpu_id', type=int, default=-1)\n    parser.add_argument('--data_dir', dest='data_dir', type=str, default='')\n    parser.add_argument('--manualSeed', type=int, help='manual seed')\n    args = parser.parse_args()\n    return args\n    "

In [4]:
args = easydict.EasyDict({
    "cfg_file": 'cfg/coco_attn2.yml',
    "gpu_id": 0,
    "data_dir": '../data/coco',
    "manualSeed": 100
})

In [5]:
def gen_example(wordtoix, algo):
    '''generate images from example sentences'''
    from nltk.tokenize import RegexpTokenizer
    filepath = '%s/example_filenames.txt' % (cfg.DATA_DIR)
    data_dic = {}
    with open(filepath, "r") as f:
        filenames = f.read().split('\n')
        for name in filenames:
            if len(name) == 0:
                continue
            filepath = '%s/%s.txt' % (cfg.DATA_DIR, name)
            with open(filepath, "r") as f:
                print('Load from:', name)
                sentences = f.read().split('\n')
                # a list of indices for a sentence
                captions = []
                cap_lens = []
                for sent in sentences:
                    if len(sent) == 0:
                        continue
                    sent = sent.replace("\ufffd\ufffd", " ")
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(sent.lower())
                    if len(tokens) == 0:
                        print('sent', sent)
                        continue

                    rev = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0 and t in wordtoix:
                            rev.append(wordtoix[t])
                    captions.append(rev)
                    cap_lens.append(len(rev))
            max_len = np.max(cap_lens)

            sorted_indices = np.argsort(cap_lens)[::-1]
            cap_lens = np.asarray(cap_lens)
            cap_lens = cap_lens[sorted_indices]
            cap_array = np.zeros((len(captions), max_len), dtype='int64')
            for i in range(len(captions)):
                idx = sorted_indices[i]
                cap = captions[idx]
                c_len = len(cap)
                cap_array[i, :c_len] = cap
            key = name[(name.rfind('/') + 1):]
            data_dic[key] = [cap_array, cap_lens, sorted_indices]
    algo.gen_example(data_dic)

In [6]:
if __name__ == "__main__":
    args #= parse_args()
    if args.cfg_file is not None:
        cfg_from_file(args.cfg_file)

    if args.gpu_id != -1:
        cfg.GPU_ID = args.gpu_id
    else:
        cfg.CUDA = False

    if args.data_dir != '':
        cfg.DATA_DIR = args.data_dir
    print('Using config:')
    pprint.pprint(cfg)

    if not cfg.TRAIN.FLAG:
        args.manualSeed = 100
    elif args.manualSeed is None:
        args.manualSeed = random.randint(1, 10000)
    random.seed(args.manualSeed)
    np.random.seed(args.manualSeed)
    torch.manual_seed(args.manualSeed)
    if cfg.CUDA:
        torch.cuda.manual_seed_all(args.manualSeed)

    now = datetime.datetime.now(dateutil.tz.tzlocal())
    timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
    output_dir = '../output/%s_%s_%s' % \
        (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

    split_dir, bshuffle = 'train', True
    if not cfg.TRAIN.FLAG:
        # bshuffle = False
        split_dir = 'test'

Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'glu-gan2',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 96,
         'GF_DIM': 48,
         'R_NUM': 3,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 10, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 12},
 'TRAIN': {'BATCH_SIZE': 40,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.0002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 150,
           'NET_E': '../DAMSMencoders/coco/text_encoder100.pth',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 50.0},
           'SNAPSHOT_INTERVAL': 10},
 'TREE': {'BASE_SIZE': 64, 'BRANCH_NUM': 3},
 

D:\Jupyter\meseong\2020.10.01_10caption_aug\code\miscc\config.py:103: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [7]:
# Get data loader
    imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM - 1))
    image_transform = transforms.Compose([
        transforms.Resize(int(imsize * 76 / 64)),
        transforms.RandomCrop(imsize),
        transforms.RandomHorizontalFlip()])
    dataset = TextDataset(cfg.DATA_DIR, split_dir,
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
    assert dataset
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=cfg.TRAIN.BATCH_SIZE,
        drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40504)
Load from:  ../data/coco\captions.pickle


In [8]:
warnings.filterwarnings(action='ignore')

In [ ]:
# Define models and go to train/evaluate
    algo = trainer(output_dir, dataloader, dataset.n_words, dataset.ixtoword)

    start_t = time.time()
    if cfg.TRAIN.FLAG:
        algo.train()
    else:
        '''generate images from pre-extracted embeddings'''
        if cfg.B_VALIDATION:
            algo.sampling(split_dir)  # generate images for the whole valid dataset
        else:
            gen_example(dataset.wordtoix, algo)  # generate images for customized captions
    end_t = time.time()
    print('Total time for training:', end_t - start_t)

Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Load image encoder from: ../DAMSMencoders/coco/image_encoder100.pth
Load text encoder from: ../DAMSMencoders/coco/text_encoder100.pth
# of netsD 3
errD0: 0.54 errD1: 0.52 errD2: 0.55 
g_loss0: 10.68 g_loss1: 14.05 g_loss2: 14.03 w_loss: 443.61 s_loss: 333.67 kl_loss: 0.00 
errD0: 0.44 errD1: 0.36 errD2: 0.52 
g_loss0: 11.59 g_loss1: 12.59 g_loss2: 12.04 w_loss: 346.96 s_loss: 307.23 kl_loss: 0.00 
errD0: 0.11 errD1: 0.11 errD2: 0.31 
g_loss0: 15.78 g_loss1: 12.14 g_loss2: 13.84 w_loss: 238.36 s_loss: 249.18 kl_loss: 0.00 
errD0: 0.29 errD1: 0.22 errD2: 0.40 
g_loss0: 15.15 g_loss1: 15.93 g_loss2: 17.02 w_loss: 184.57 s_loss: 203.82 kl_loss: 0.00 
errD0: 0.60 errD1: 0.34 errD2: 0.41 
g_loss0: 8.31 g_loss1: 5.98 g_loss2: 11.60 w_loss: 214.63 s_loss: 179.02 kl_loss: 0.00 
errD0: 0.01 errD1: 0.03 errD2: 0.03 
g_loss0: 15.47 g_loss1: 14.29 g_loss2: 15.96 w_loss: 142.34 s_loss: 151.63 kl_loss: 0.

errD0: 0.06 errD1: 0.05 errD2: 0.04 
g_loss0: 14.33 g_loss1: 10.69 g_loss2: 18.99 w_loss: 24.34 s_loss: 61.58 kl_loss: 0.03 
errD0: 0.02 errD1: 0.02 errD2: 0.01 
g_loss0: 11.91 g_loss1: 16.29 g_loss2: 14.87 w_loss: 35.61 s_loss: 50.69 kl_loss: 0.03 
errD0: 0.07 errD1: 0.02 errD2: 0.05 
g_loss0: 12.43 g_loss1: 15.86 g_loss2: 15.67 w_loss: 36.02 s_loss: 63.92 kl_loss: 0.04 
errD0: 0.03 errD1: 0.03 errD2: 0.07 
g_loss0: 9.74 g_loss1: 6.72 g_loss2: 12.70 w_loss: 25.84 s_loss: 62.81 kl_loss: 0.04 
errD0: 0.01 errD1: 0.04 errD2: 0.05 
g_loss0: 10.96 g_loss1: 11.53 g_loss2: 13.07 w_loss: 25.92 s_loss: 59.23 kl_loss: 0.03 
errD0: 0.04 errD1: 0.04 errD2: 0.06 
g_loss0: 18.32 g_loss1: 15.20 g_loss2: 12.13 w_loss: 19.17 s_loss: 45.95 kl_loss: 0.03 
errD0: 0.11 errD1: 0.05 errD2: 0.11 
g_loss0: 7.02 g_loss1: 9.96 g_loss2: 12.57 w_loss: 21.33 s_loss: 54.47 kl_loss: 0.04 
errD0: 0.09 errD1: 0.06 errD2: 0.03 
g_loss0: 10.86 g_loss1: 26.73 g_loss2: 14.40 w_loss: 17.36 s_loss: 49.49 kl_loss: 0.02 
errD

errD0: 0.01 errD1: 0.02 errD2: 0.09 
g_loss0: 16.26 g_loss1: 11.96 g_loss2: 17.00 w_loss: 22.35 s_loss: 47.49 kl_loss: 0.08 
errD0: 0.01 errD1: 0.11 errD2: 0.16 
g_loss0: 20.60 g_loss1: 10.99 g_loss2: 14.11 w_loss: 17.31 s_loss: 41.31 kl_loss: 0.13 
errD0: 0.01 errD1: 0.01 errD2: 0.00 
g_loss0: 12.86 g_loss1: 18.61 g_loss2: 13.21 w_loss: 15.14 s_loss: 45.67 kl_loss: 0.11 
errD0: 0.00 errD1: 0.06 errD2: 0.00 
g_loss0: 13.89 g_loss1: 12.43 g_loss2: 18.15 w_loss: 30.17 s_loss: 43.91 kl_loss: 0.09 
errD0: 0.03 errD1: 0.49 errD2: 0.08 
g_loss0: 15.67 g_loss1: 1.73 g_loss2: 7.51 w_loss: 14.71 s_loss: 30.71 kl_loss: 0.10 
errD0: 0.02 errD1: 0.11 errD2: 0.01 
g_loss0: 12.25 g_loss1: 11.31 g_loss2: 10.70 w_loss: 19.90 s_loss: 42.25 kl_loss: 0.12 
errD0: 1.22 errD1: 1.96 errD2: 0.22 
g_loss0: 7.30 g_loss1: 40.26 g_loss2: 13.86 w_loss: 24.27 s_loss: 51.27 kl_loss: 0.12 
errD0: 0.07 errD1: 0.02 errD2: 0.02 
g_loss0: 16.38 g_loss1: 29.50 g_loss2: 18.79 w_loss: 15.28 s_loss: 46.56 kl_loss: 0.08 
err

errD0: 0.03 errD1: 0.01 errD2: 0.09 
g_loss0: 10.68 g_loss1: 13.54 g_loss2: 20.90 w_loss: 15.96 s_loss: 39.45 kl_loss: 0.16 
errD0: 0.01 errD1: 0.01 errD2: 0.02 
g_loss0: 14.44 g_loss1: 10.54 g_loss2: 11.34 w_loss: 20.20 s_loss: 38.93 kl_loss: 0.16 
errD0: 0.04 errD1: 0.02 errD2: 0.26 
g_loss0: 12.81 g_loss1: 12.72 g_loss2: 7.56 w_loss: 19.03 s_loss: 34.13 kl_loss: 0.12 
errD0: 0.01 errD1: 0.01 errD2: 0.26 
g_loss0: 27.11 g_loss1: 13.13 g_loss2: 13.45 w_loss: 24.16 s_loss: 45.43 kl_loss: 0.15 
errD0: 0.40 errD1: 2.11 errD2: 0.01 
g_loss0: 30.68 g_loss1: 26.43 g_loss2: 17.48 w_loss: 18.04 s_loss: 40.27 kl_loss: 0.17 
errD0: 0.02 errD1: 0.07 errD2: 0.02 
g_loss0: 12.08 g_loss1: 14.13 g_loss2: 7.97 w_loss: 18.89 s_loss: 45.10 kl_loss: 0.13 
errD0: 0.24 errD1: 0.02 errD2: 0.09 
g_loss0: 6.26 g_loss1: 22.56 g_loss2: 4.58 w_loss: 19.96 s_loss: 41.76 kl_loss: 0.16 
errD0: 0.12 errD1: 0.06 errD2: 0.09 
g_loss0: 18.81 g_loss1: 23.87 g_loss2: 9.75 w_loss: 19.65 s_loss: 46.70 kl_loss: 0.17 
errD0

errD0: 0.09 errD1: 0.01 errD2: 0.11 
g_loss0: 15.13 g_loss1: 15.28 g_loss2: 12.77 w_loss: 15.23 s_loss: 40.88 kl_loss: 0.18 
errD0: 0.04 errD1: 1.17 errD2: 0.12 
g_loss0: 19.17 g_loss1: 1.22 g_loss2: 12.32 w_loss: 10.70 s_loss: 24.49 kl_loss: 0.25 
errD0: 0.01 errD1: 0.01 errD2: 0.16 
g_loss0: 15.70 g_loss1: 14.49 g_loss2: 27.17 w_loss: 18.25 s_loss: 42.02 kl_loss: 0.19 
errD0: 0.02 errD1: 0.01 errD2: 0.03 
g_loss0: 11.58 g_loss1: 18.11 g_loss2: 11.04 w_loss: 12.44 s_loss: 33.00 kl_loss: 0.21 
errD0: 0.01 errD1: 0.01 errD2: 0.25 
g_loss0: 18.54 g_loss1: 15.81 g_loss2: 7.60 w_loss: 12.85 s_loss: 30.43 kl_loss: 0.25 
errD0: 0.01 errD1: 0.08 errD2: 0.02 
g_loss0: 14.01 g_loss1: 12.10 g_loss2: 15.68 w_loss: 17.96 s_loss: 28.84 kl_loss: 0.24 
errD0: 0.02 errD1: 0.01 errD2: 0.08 
g_loss0: 15.49 g_loss1: 17.02 g_loss2: 13.79 w_loss: 27.74 s_loss: 55.85 kl_loss: 0.19 
errD0: 0.01 errD1: 0.01 errD2: 0.00 
g_loss0: 13.76 g_loss1: 11.26 g_loss2: 19.66 w_loss: 15.96 s_loss: 37.59 kl_loss: 0.23 
er

...

errD0: 0.01 errD1: 0.00 errD2: 0.00 
g_loss0: 21.50 g_loss1: 17.11 g_loss2: 17.48 w_loss: 13.27 s_loss: 30.72 kl_loss: 0.64 
errD0: 0.11 errD1: 0.02 errD2: 0.00 
g_loss0: 18.67 g_loss1: 12.15 g_loss2: 25.61 w_loss: 6.10 s_loss: 27.09 kl_loss: 0.59 
errD0: 0.01 errD1: 0.04 errD2: 0.02 
g_loss0: 14.12 g_loss1: 27.47 g_loss2: 16.64 w_loss: 11.91 s_loss: 26.85 kl_loss: 0.60 
errD0: 0.00 errD1: 0.01 errD2: 0.00 
g_loss0: 25.17 g_loss1: 12.75 g_loss2: 21.16 w_loss: 9.28 s_loss: 32.46 kl_loss: 0.65 
errD0: 0.02 errD1: 0.02 errD2: 0.02 
g_loss0: 16.82 g_loss1: 11.63 g_loss2: 13.75 w_loss: 7.09 s_loss: 25.16 kl_loss: 0.63 
errD0: 0.04 errD1: 0.01 errD2: 0.00 
g_loss0: 13.21 g_loss1: 17.08 g_loss2: 18.72 w_loss: 13.35 s_loss: 31.50 kl_loss: 0.60 
errD0: 0.00 errD1: 0.00 errD2: 0.00 
g_loss0: 16.52 g_loss1: 21.37 g_loss2: 13.54 w_loss: 15.65 s_loss: 30.31 kl_loss: 0.50 
errD0: 0.01 errD1: 0.00 errD2: 0.00 
g_loss0: 16.02 g_loss1: 26.03 g_loss2: 18.01 w_loss: 16.13 s_loss: 34.06 kl_loss: 0.60 
err

errD0: 0.01 errD1: 0.02 errD2: 0.00 
g_loss0: 17.24 g_loss1: 6.77 g_loss2: 21.80 w_loss: 10.19 s_loss: 26.66 kl_loss: 0.62 
errD0: 0.01 errD1: 0.10 errD2: 0.01 
g_loss0: 17.15 g_loss1: 18.08 g_loss2: 13.52 w_loss: 10.81 s_loss: 31.57 kl_loss: 0.56 
errD0: 0.02 errD1: 0.03 errD2: 0.00 
g_loss0: 18.19 g_loss1: 12.55 g_loss2: 24.98 w_loss: 6.93 s_loss: 23.06 kl_loss: 0.74 
errD0: 0.00 errD1: 0.00 errD2: 0.01 
g_loss0: 16.41 g_loss1: 27.47 g_loss2: 12.17 w_loss: 10.53 s_loss: 30.54 kl_loss: 0.51 
errD0: 0.01 errD1: 0.01 errD2: 0.00 
g_loss0: 17.10 g_loss1: 18.53 g_loss2: 18.21 w_loss: 14.66 s_loss: 36.65 kl_loss: 0.51 
errD0: 0.02 errD1: 0.00 errD2: 0.00 
g_loss0: 20.26 g_loss1: 21.44 g_loss2: 14.62 w_loss: 13.04 s_loss: 28.79 kl_loss: 0.66 
errD0: 0.00 errD1: 0.01 errD2: 0.13 
g_loss0: 26.91 g_loss1: 28.72 g_loss2: 24.67 w_loss: 8.12 s_loss: 30.46 kl_loss: 0.56 
errD0: 0.00 errD1: 0.00 errD2: 0.00 
g_loss0: 20.37 g_loss1: 15.58 g_loss2: 18.18 w_loss: 8.46 s_loss: 23.44 kl_loss: 0.83 
errD

errD0: 0.45 errD1: 0.13 errD2: 0.01 
g_loss0: 42.97 g_loss1: 20.20 g_loss2: 18.55 w_loss: 10.28 s_loss: 27.09 kl_loss: 0.78 
errD0: 0.00 errD1: 0.00 errD2: 0.00 
g_loss0: 25.21 g_loss1: 16.34 g_loss2: 13.71 w_loss: 10.50 s_loss: 36.75 kl_loss: 0.65 
errD0: 0.00 errD1: 0.00 errD2: 0.00 
g_loss0: 22.78 g_loss1: 27.90 g_loss2: 21.65 w_loss: 10.26 s_loss: 29.94 kl_loss: 0.69 
errD0: 0.00 errD1: 0.01 errD2: 0.00 
g_loss0: 17.01 g_loss1: 14.28 g_loss2: 22.19 w_loss: 13.56 s_loss: 32.16 kl_loss: 0.71 
errD0: 0.02 errD1: 0.00 errD2: 0.01 
g_loss0: 13.62 g_loss1: 16.65 g_loss2: 28.69 w_loss: 10.90 s_loss: 22.42 kl_loss: 0.67 
errD0: 0.00 errD1: 0.00 errD2: 0.07 
g_loss0: 21.31 g_loss1: 18.25 g_loss2: 14.32 w_loss: 20.24 s_loss: 43.01 kl_loss: 0.69 
errD0: 0.02 errD1: 0.01 errD2: 0.00 
g_loss0: 29.18 g_loss1: 13.84 g_loss2: 19.92 w_loss: 13.86 s_loss: 33.25 kl_loss: 0.60 
errD0: 0.01 errD1: 0.00 errD2: 0.04 
g_loss0: 15.54 g_loss1: 18.91 g_loss2: 21.65 w_loss: 14.40 s_loss: 40.87 kl_loss: 0.63 


errD0: 0.00 errD1: 0.02 errD2: 0.00 
g_loss0: 22.70 g_loss1: 12.53 g_loss2: 22.38 w_loss: 8.28 s_loss: 25.11 kl_loss: 0.60 
errD0: 0.04 errD1: 0.00 errD2: 0.01 
g_loss0: 17.76 g_loss1: 13.65 g_loss2: 19.76 w_loss: 13.51 s_loss: 41.23 kl_loss: 0.70 
errD0: 0.02 errD1: 0.02 errD2: 3.60 
g_loss0: 17.42 g_loss1: 15.84 g_loss2: 49.18 w_loss: 13.42 s_loss: 35.87 kl_loss: 0.46 
errD0: 0.00 errD1: 0.02 errD2: 0.00 
g_loss0: 25.22 g_loss1: 22.87 g_loss2: 20.81 w_loss: 11.64 s_loss: 28.88 kl_loss: 0.71 
errD0: 0.04 errD1: 0.02 errD2: 0.01 
g_loss0: 12.55 g_loss1: 17.04 g_loss2: 28.98 w_loss: 21.09 s_loss: 32.83 kl_loss: 0.77 
errD0: 0.00 errD1: 0.01 errD2: 0.01 
g_loss0: 30.13 g_loss1: 20.32 g_loss2: 17.16 w_loss: 13.33 s_loss: 24.40 kl_loss: 0.57 
errD0: 0.00 errD1: 0.00 errD2: 0.03 
g_loss0: 16.53 g_loss1: 14.90 g_loss2: 12.98 w_loss: 11.03 s_loss: 39.03 kl_loss: 0.59 
errD0: 0.00 errD1: 0.12 errD2: 0.00 
g_loss0: 18.07 g_loss1: 19.43 g_loss2: 14.56 w_loss: 10.44 s_loss: 30.33 kl_loss: 0.59 
e

errD0: 0.00 errD1: 0.16 errD2: 0.01 
g_loss0: 20.95 g_loss1: 13.07 g_loss2: 11.54 w_loss: 9.48 s_loss: 23.78 kl_loss: 0.59 
errD0: 0.30 errD1: 0.01 errD2: 3.97 
g_loss0: 13.64 g_loss1: 23.10 g_loss2: 0.00 w_loss: 20.85 s_loss: 40.18 kl_loss: 0.85 
errD0: 0.00 errD1: 0.00 errD2: 0.01 
g_loss0: 23.09 g_loss1: 17.67 g_loss2: 12.86 w_loss: 16.81 s_loss: 34.37 kl_loss: 0.50 
errD0: 0.01 errD1: 0.03 errD2: 0.01 
g_loss0: 13.99 g_loss1: 13.08 g_loss2: 22.28 w_loss: 15.38 s_loss: 39.91 kl_loss: 0.73 
errD0: 0.00 errD1: 0.00 errD2: 0.04 
g_loss0: 23.14 g_loss1: 18.79 g_loss2: 34.69 w_loss: 14.25 s_loss: 34.80 kl_loss: 0.74 
errD0: 0.00 errD1: 0.17 errD2: 0.08 
g_loss0: 23.29 g_loss1: 19.86 g_loss2: 18.57 w_loss: 11.93 s_loss: 31.40 kl_loss: 0.74 
errD0: 0.00 errD1: 0.00 errD2: 0.00 
g_loss0: 22.47 g_loss1: 24.86 g_loss2: 17.20 w_loss: 24.21 s_loss: 40.14 kl_loss: 0.63 
errD0: 0.02 errD1: 0.00 errD2: 0.00 
g_loss0: 14.21 g_loss1: 16.96 g_loss2: 24.20 w_loss: 14.41 s_loss: 34.85 kl_loss: 0.83 
[8